___
# Artificial Neural Network with Keras 

In this lab you will build a **ANN** (Perceptron/MLP) for Loan Prediction (predict whether a loan will or will not be given).
___

## 1. Loading the Dataset & Inspecting


For this Lab, I have used a subset of the Loan Prediction dataset. You can download the training and testing dataset from here: [Download Data](https://drive.google.com/drive/folders/10beInU2j3J9tF_dpbt50_36oxoa9jJQx?usp=sharing)

Note : Testing data that you are provided is the subset of the training data from Loan Prediction problem.

 

Now, lets get started by importing important packages and the dataset.

In [2]:
# Load python modules

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import model_selection


In [3]:
# Load dataset from local drive (for colab notebook)
from google.colab import files
import io

uploaded = files.upload()    # Will prompt you to select the training file with the features and its values
train_dataset = pd.read_csv(io.BytesIO(uploaded['X_train.csv']))

Saving X_train.csv to X_train.csv


In [71]:
uploaded = files.upload()    # Will prompt you to select the training file with the labels
train_labels = pd.read_csv(io.BytesIO(uploaded['Y_train.csv']))

Saving Y_train.csv to Y_train (3).csv


In [5]:
uploaded = files.upload()    # Will prompt you to select the testing file with the features and its values
test_dataset = pd.read_csv(io.BytesIO(uploaded['X_test.csv']))

Saving X_test.csv to X_test.csv


In [72]:
uploaded = files.upload()    # Will prompt you to select the testing file with the labels
test_labels = pd.read_csv(io.BytesIO(uploaded['Y_test.csv']))

Saving Y_test.csv to Y_test (3).csv


In [ ]:
# shape
print(train_dataset.shape)
print(train_labels.shape)
train_dataset

In [ ]:
print(test_dataset.shape)
print(test_labels.shape)

In [ ]:
# list of column titles 
print(train_dataset.columns)
print(train_labels.columns)

In [ ]:
# list of column (field) data types
print(train_dataset.dtypes)
print(train_labels.dtypes)

In [ ]:
# head
train_dataset.head(8)


In [ ]:
# create a bar plot: 
# A bar plot is a plot that presents categorical data with rectangular bars with lengths proportional to the values that they represent. 
# A bar plot shows comparisons among discrete categories.
train_dataset['Gender'].value_counts().plot(kind='bar')



In [ ]:
train_dataset['Married'].value_counts().plot(kind='bar')


In [ ]:
train_dataset['Dependents'].value_counts().plot(kind='bar')


In [ ]:
train_dataset['Education'].value_counts().plot(kind='bar')


In [ ]:
train_dataset['Self_Employed'].value_counts().plot(kind='bar')


In [ ]:
train_dataset['Property_Area'].value_counts().plot(kind='bar')


In [ ]:
# Plot a histogram; it returns one histogram per numerical column
# It is a representation of the distribution of data
train_dataset.hist()


##2. Managing Missing Data

Sometimes you may find some data are missing in the dataset. If the missing values are not handled properly inaccurate inference about the data may result. Due to improper handling, the result obtained will differ from ones where the missing values are present. 

Since missing values can tangibly reduce prediction accuracy, this step needs to be a priority. In terms of machine learning, assumed or approximated values are “more appropriate” for an algorithm than just missing ones.  Even if you don’t know the exact value, methods exist to better “assume” which value is missing or bypass the issue. So how to сlean the data here? Choosing the right approach also heavily depends on data and the domain you have:
* Substitute missing values with dummy values, e.g. n/a for categorical or 0 for numerical values.
* Substitute the missing numerical values with mean figures.
* For categorical values, you can also use the most frequent items to fill in.

###2.1 Remove Unnecessary Features/Columns from both the training and testing dataset


In [ ]:
train_dataset = train_dataset.drop("Loan_ID", axis=1)
train_dataset

In [ ]:
test_dataset = test_dataset.drop("Loan_ID", axis=1)
test_dataset_no_nans = test_dataset.copy()
test_dataset_no_nans

### 2.2 Filling in Missing Data



In [ ]:
# check which features contain missing data/NaN values
print(train_dataset.isnull().sum())



In [ ]:
# handling missing data for numeric features
from sklearn.impute import SimpleImputer 

train_dataset_no_nans =  train_dataset.copy()

# 1. Imputer
imptr = SimpleImputer(missing_values = np.nan, strategy = 'mean')  


# 2. Fit the imputer object to the feature matrix (only for numeric features)
imptr = imptr.fit(train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome',
                'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']] = \
imptr.transform(train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term', 'Credit_History']]) 

train_dataset_no_nans



In [ ]:
# handling missing data for categorical features

# 1. Imputer
imptr = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')  

# 2. Fit the imputer object to the feature matrix (both for numeric/categorical features)
imptr = imptr.fit(train_dataset_no_nans[['Gender', 'Married', 'Dependents', 'Education', 'Property_Area']])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[['Gender', 'Married', 'Dependents', 'Education', 'Property_Area']] = \
imptr.transform(train_dataset_no_nans[['Gender', 'Married', 'Dependents', 'Education', 'Property_Area']]) 

train_dataset_no_nans





In [ ]:
print(train_dataset_no_nans.isnull().sum())


One should follow the same procedure for filling in the missing data in the testing set, as well. However, in our case, the testing set does not contain any missing data.

##3.  Categorical Data

Data Preprocessing in machine learning requires values of the data in numerical form. Therefore text values in the columns of datasets must be converted into numerical form. 

###3.1 Converting categorical to numerical values

Given the original dataset, it is clear we have a few categorical features. All these need to be encoded. The [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class is used to transform the categorical or string values to numerical ones (between 0 and n_classes-1).



In [23]:
# we convert the features that you see below by using the label encoder as they have binary values
from sklearn.preprocessing import LabelEncoder
lblEncoder_X = LabelEncoder()                 # create an object of the LabelEncoder class
train_dataset_no_nans['Gender'] = lblEncoder_X.fit_transform(train_dataset_no_nans['Gender']) 
train_dataset_no_nans['Married'] = lblEncoder_X.fit_transform(train_dataset_no_nans['Married']) 
train_dataset_no_nans['Education'] = lblEncoder_X.fit_transform(train_dataset_no_nans['Education']) 
train_dataset_no_nans['Self_Employed'] = lblEncoder_X.fit_transform(train_dataset_no_nans['Self_Employed']) 




In [ ]:
# encode categorical data for the testing set too!
from sklearn.preprocessing import LabelEncoder
lblEncoder_X = LabelEncoder()                 # create an object of the LabelEncoder class
test_dataset_no_nans['Gender'] = lblEncoder_X.fit_transform(test_dataset_no_nans['Gender']) 
test_dataset_no_nans['Married'] = lblEncoder_X.fit_transform(test_dataset_no_nans['Married']) 
test_dataset_no_nans['Education'] = lblEncoder_X.fit_transform(test_dataset_no_nans['Education']) 
test_dataset_no_nans['Self_Employed'] = lblEncoder_X.fit_transform(test_dataset_no_nans['Self_Employed']) 



print(test_dataset_no_nans)



Now let us try a different encoding strategy, the one-hot encoding for the categorical features that are non-binary (contain at least 3 different values)

In [ ]:
from sklearn.preprocessing import OneHotEncoder


oneHotEncoder = OneHotEncoder()                                      # create OneHotEncoder object 
aaa = oneHotEncoder.fit(train_dataset_no_nans[['Property_Area']])           # fit the OneHotEncoder object to feature Gender
print(aaa.categories_)

dum = aaa.transform(train_dataset_no_nans[['Property_Area']]).toarray()               

train_dataset_no_nans['Rural'] = dum[:,0]     
train_dataset_no_nans['Semiurban'] = dum[:,1]       
train_dataset_no_nans['Urban'] = dum[:,2]       

train_dataset_no_nans= train_dataset_no_nans.drop(columns='Property_Area',axis=1)  

######
aaa = oneHotEncoder.fit(train_dataset_no_nans[['Dependents']])           
print(aaa.categories_)

dum = aaa.transform(train_dataset_no_nans[['Dependents']]).toarray()               


train_dataset_no_nans['0'] = dum[:,0]     
train_dataset_no_nans['1'] = dum[:,1]       
train_dataset_no_nans['2'] = dum[:,2]       
train_dataset_no_nans['3+'] = dum[:,3]       


train_dataset_no_nans= train_dataset_no_nans.drop(columns='Dependents',axis=1)  
train_dataset_no_nans

In [ ]:
# one-hot encoding for the non-binary categorical data for the testing set too!

oneHotEncoder = OneHotEncoder()                                      # create OneHotEncoder object 
aaa = oneHotEncoder.fit(test_dataset_no_nans[['Property_Area']])           # fit the OneHotEncoder object to feature Gender
print(aaa.categories_)

dum = aaa.transform(test_dataset_no_nans[['Property_Area']]).toarray()               

test_dataset_no_nans['Rural'] = dum[:,0]     
test_dataset_no_nans['Semiurban'] = dum[:,1]       
test_dataset_no_nans['Urban'] = dum[:,2]      

test_dataset_no_nans= test_dataset_no_nans.drop(columns='Property_Area',axis=1)  

########

aaa = oneHotEncoder.fit(test_dataset_no_nans[['Dependents']])           # fit the OneHotEncoder object to feature Gender
print(aaa.categories_)

dum = aaa.transform(test_dataset_no_nans[['Dependents']]).toarray()               


test_dataset_no_nans['0'] = dum[:,0]     
test_dataset_no_nans['1'] = dum[:,1]      
test_dataset_no_nans['2'] = dum[:,2]       
test_dataset_no_nans['3+'] = dum[:,3]       


test_dataset_no_nans= test_dataset_no_nans.drop(columns='Dependents',axis=1)  

test_dataset_no_nans

##4.  Compute Statistics, Check Imbalance, Convert Categorical Labels to Numeric Ones

In [ ]:
import matplotlib.pyplot as plt

# compute histograms of numeric features (you can see the ranges)
train_dataset_no_nans[train_dataset_no_nans.dtypes[(train_dataset_no_nans.dtypes=="float64") | (train_dataset_no_nans.dtypes=="int64")].index.values].hist(figsize=[15,15])  # the symbol '|' means 'or'



In [ ]:
# same for the testing set
test_dataset_no_nans[test_dataset_no_nans.dtypes[(test_dataset_no_nans.dtypes=="float64") | (test_dataset_no_nans.dtypes=="int64")].index.values].hist(figsize=[15,15])  # the symbol '|' means 'or'


In [ ]:
sampleClassBias = train_labels['Target'].value_counts()
print('Training Labels distribution:')
print(sampleClassBias)

sampleClassBias = test_labels['Target'].value_counts()
print('Test Labels distribution:')
print(sampleClassBias)

In [ ]:
lblEncoder_X = LabelEncoder()                 # create an object of the LabelEncoder class
train_labels['Target'] = lblEncoder_X.fit_transform(train_labels['Target']) 
test_labels['Target'] = lblEncoder_X.fit_transform(test_labels['Target']) 
train_labels


##5.  Feature Scaling

When the data is comprised of feature values with varying scales, many machine learning algorithms can benefit from rescaling the attributes to all have the same scale. 


###5.1 Implementing Feature Scaling

Feature scaling is a method used to scale the range of variables/values of features. 

>![feature scaling](https://drive.google.com/uc?id=1loaMbVo_7ZJo53Ogv7I0XMG_wSOOYBxI)

>  Figure 5: Feature Scaling methods
> Where X is the observation feature.

**It is not essential to understand the math behind these methods, what is important is to remember that the variables are being put in the same range / same scale so that no variable is dominated by another.**


In [ ]:
#normalisation
train_dataset_no_nans_normalised = train_dataset_no_nans.copy()


# Importing MinMaxScaler and initializing it
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
# Normalising
train_dataset_no_nans_normalised[:]=min_max.fit_transform(train_dataset_no_nans_normalised[:])
train_dataset_no_nans_normalised


In [ ]:
#normalisation
test_dataset_no_nans_normalised = test_dataset_no_nans.copy()


# Importing MinMaxScaler and initializing it
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
# Normalising
test_dataset_no_nans_normalised[:]=min_max.fit_transform(test_dataset_no_nans_normalised[:])
test_dataset_no_nans_normalised


Question: Do we need to apply feature scaling to the labels? The answer is no! The label is a categorical value that takes 2 values either no (0) or yes (1).  It is a classification problem.


In [ ]:
# Standardisation
train_dataset_no_nans_standardised = train_dataset_no_nans.copy()

#Standardizing 
from sklearn.preprocessing import scale
train_dataset_no_nans_standardised[:]=scale(train_dataset_no_nans_standardised[:])
train_dataset_no_nans_standardised

In [ ]:
# Standardisation
test_dataset_no_nans_standardised = test_dataset_no_nans.copy()

#Standardizing 
test_dataset_no_nans_standardised[:]=scale(test_dataset_no_nans_standardised[:])
test_dataset_no_nans_standardised

##7. Use KNN classification method



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Normalized data
# We will use a classifier (we will skip details and what it does for now, so just use it as it is)
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(train_dataset_no_nans_normalised, train_labels['Target'])  
# Checking the model's accuracy (performance)
print(accuracy_score(test_labels['Target'],knn.predict(test_dataset_no_nans_normalised)))

# Standardized data
knn.fit(train_dataset_no_nans_standardised, train_labels['Target'])  
print(accuracy_score(test_labels['Target'],knn.predict(test_dataset_no_nans_standardised)))

##  Build ANN classifier and Test Performance


In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

import numpy as np

%matplotlib inline

model = models.Sequential()
model.add(layers.Dense(20, input_dim=16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


# as metric we choose the accuracy: the total number of correct predictions made
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

# Training
history = model.fit(train_dataset_no_nans_standardised, train_labels, epochs=50, batch_size=130, 
                    validation_data=(test_dataset_no_nans_standardised, test_labels))

# Validation
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')



## Try/Check the following (in random order)

1) Standardization vs Normalization </br>
2) Adding more hidden units to the network </br>
3) Adding more layers to the network </br>
4) Using different activation functions in the hidden units </br>
5) Using different optimizers </br>
6) Using different learning rates </br>
7) Train for more epochs </br>


## More advanced:

1) Check other metrics. </br>
2) Find accuracy per category (meaning the accuracy when train_labels['Target'] = 1 and accuracy when train_labels['Target'] = 0) </br>
3) For the labels we used the LabelEncoder; what if we performed one-hot encoding. What modifications should we do in the code of the section 'Build ANN classifier and Test Performance'?  </br>
Note: no need to actually perform one-hot encoding to the labels and thus the code performing the LabelEncoder in Section 4 (Compute Statistics, Check Imbalance, Convert Categorical Labels to Numeric Ones) should not be changed. Modification should be made in the code of the section 'Build ANN classifier and Test Performance'. </br> 
What is the performance of the ANN in that case? </br>


